# Post Boot Tasks with Templating

This Jupyter notebook will walk you through how to use threads to execute commands that configure your experiment. 

## Import the FABlib Library

The `show_config` method shows what some environment variables relevant to your FABRIC user are set to. All necessary variables may be set [here](../../../fabric_examples/fablib_api/configure_environment/configure_environment.ipynb).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following creates a set of nodes with basic compute capabilities on a random site. 

The notable steps in this cell are the calls to `add_post_boot_upload_file`, `add_post_boot_upload_directory`, and `add_post_boot_execute`.  These functions are the equivalent of their respective counterparts (`upload_file`, `upload_directory`, and `execute`) that can be used after the slices are active.  The difference is that you can add the post boot tasks before you submit the slice.  Adding a post boot task appends the task to a list of tasks that will be executed in order during the post boot config step. Additionally, the task list for each node is executed in parallel.

Specifically, this example copies a directory called `node_tools` from the notebook’s folder to each of the VMs.  It then executes a `config_script.sh` script that is located in the directory.  By default, the script calls `dnf` to install `tcpdump` and create a file.  However, feel free to edit the script to your preference.


In [ ]:
slice_name="MySlice2"

site=fablib.get_random_site()

#Create a slice
slice = fablib.new_slice(name=slice_name)


for i in range(4):
    # Add a node
    node = slice.add_node(name=f"Node{i}", site=site)
    node.add_fabnet()
    
    iface = node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}')
    
    node.add_post_boot_upload_directory('node_tools','.')
    node.add_post_boot_execute(f"chmod +x node_tools/config_script.sh && ./node_tools/config_script.sh {{{{ interfaces['{iface.get_name()}'].dev }}}}  ")

#Submit the Request
slice.submit()

In [ ]:
for node in slice.get_nodes():
    stdout, stderr = node.execute('echo -n `hostname -s`": " && cat post_boot_output.txt')


## Template Options

You can find the options for template parameters by printing the template context for any FABlib obect.

In [ ]:
import json
node = slice.get_node('Node1')

print(json.dumps(node.get_template_context(), indent=4))    


## Testing Templates

You can test a template by calling passing a template to the `render_template` function on any FABlib object./

In [ ]:
import json
node = slice.get_node('Node1')

iface = node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}')

print(node.render_template(f"{{{{ interfaces['{iface.get_name()}'].dev }}}}"))



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()